### 1. Using HuggingFace Transformers

In [1]:
from transformers import AutoTokenizer, AutoModel
import torch
# Sentences we want sentence embeddings for
sentences = ["酒精的危害", "如何戒烟"]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5')
model.eval()



(…)-v1.5/resolve/main/tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

(…)bge-large-zh-v1.5/resolve/main/vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

(…)arge-zh-v1.5/resolve/main/tokenizer.json:   0%|          | 0.00/439k [00:00<?, ?B/s]

(…)1.5/resolve/main/special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

(…)e-large-zh-v1.5/resolve/main/config.json:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

[2023-11-15 05:29:31,565] [INFO] [real_accelerator.py:133:get_accelerator] Setting ds_accelerator to cuda (auto detect)


model.safetensors:   0%|          | 0.00/1.30G [00:00<?, ?B/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings: tensor([[-0.0171,  0.0142, -0.0339,  ..., -0.0327, -0.0134,  0.0421],
        [ 0.0244, -0.0102,  0.0472,  ...,  0.0064,  0.0487, -0.0028]])


In [2]:
# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    # Perform pooling. In this case, cls pooling.
    sentence_embeddings = model_output[0][:, 0]
# normalize embeddings
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')print("Sentence embeddings:", sentence_embeddings)

Sentence embeddings: tensor([[-0.0171,  0.0142, -0.0339,  ..., -0.0327, -0.0134,  0.0421],
        [ 0.0244, -0.0102,  0.0472,  ...,  0.0064,  0.0487, -0.0028]])


In [19]:
from transformers import AutoTokenizer, AutoModel
import torch
# Sentences we want sentence embeddings for
sentences = ["酒精的危害", "如何戒烟"]
sentences = ["酒精的危害"]


device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')


# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-large-zh-v1.5')
tokenizer.padding_side = 'right'
model = AutoModel.from_pretrained('BAAI/bge-large-zh-v1.5').half()

model.to(device) 
model.requires_grad_(False)
model.eval()


# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt').to(device)
# for s2p(short query to long passage) retrieval task, add an instruction to query (not add instruction for passages)
# encoded_input = tokenizer([instruction + q for q in queries], padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)
    sentence_embeddings = model_output[0][:, 0]
    # Perform pooling. In this case, cls pooling

# Perform pooling. In this case, max pooling.
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1).cpu().numpy()
print("Sentence embeddings:", sentence_embeddings)

--device=cuda:0


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Sentence embeddings: [[-0.01715   0.0142   -0.03387  ... -0.03262  -0.013405  0.04205 ]]


In [11]:
model_output

BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-0.3523,  0.2917, -0.6958,  ..., -0.6704, -0.2754,  0.8638],
         [-0.8564,  0.1567, -0.7944,  ..., -0.7646,  0.3071,  0.6270],
         [ 0.3352, -0.2041, -0.8535,  ..., -0.6538,  0.6909,  0.4067],
         ...,
         [-0.6616,  0.5332, -1.2158,  ..., -0.4519,  0.1106,  0.9546],
         [-0.5513,  0.6104, -1.3975,  ..., -0.6646, -0.0892,  1.0283],
         [-0.3501,  0.2922, -0.6958,  ..., -0.6694, -0.2759,  0.8633]],

        [[ 0.5356, -0.2233,  1.0332,  ...,  0.1399,  1.0654, -0.0588],
         [ 0.6255,  0.5889,  0.7495,  ...,  0.2888,  1.4053,  0.6309],
         [ 0.4128,  0.7593,  1.3486,  ...,  0.2607,  0.4419,  1.3926],
         ...,
         [ 0.0558, -0.4360,  0.6021,  ...,  0.4229,  0.0717, -0.4097],
         [ 0.5356, -0.2209,  1.0312,  ...,  0.1396,  1.0654, -0.0590],
         [ 0.5327,  0.2764,  1.0039,  ..., -0.0260,  0.4277,  0.2615]]],
       device='cuda:0', dtype=torch.float16), pooler_

In [16]:
model_output[0][:,0]

tensor([[-0.3523,  0.2917, -0.6958,  ..., -0.6704, -0.2754,  0.8638],
        [ 0.5356, -0.2233,  1.0332,  ...,  0.1399,  1.0654, -0.0588]],
       device='cuda:0', dtype=torch.float16)